# Patents

## Abstract

Protecting intellectual property through patents is a cornerstone of Western innovation. Yet, there is no global consensus on whether patents promote or stall innovation. In particular, does innovation lead to patents or do patents lead to innovation? In this project, we propose to observe how patent applications predict technological trends, new markets, and products. We build a graph database by combining a patent citations dataset and associated metadata. We then explore the resulting graph to extract key insight into major innovation trends. Finally, we dig deeper into select markets and companies to correlate patent activity with events such as product launches or company creation as well as financial indicators such as historical stock prices and total market value.

## Imports & Configuration

In [108]:
# Imports
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import requests
import io
import urllib.request
import os
import zipfile
import gzip
import unicodedata
import re
import networkx as nx
import operator
from bs4 import BeautifulSoup

In [193]:
DATA_PATH = 'data'

## Collecting Datasets, Data Cleaning, Enrichment

### NBER Patent Dataset

We begin by collecting patent data (or metadata) associated with the period of interest (1963 - 1999). This information is available from the US National Bureau of Economic Research (NBER) [here](http://www.nber.org/patents/).

We collect and combine the different datasets provided by NBER, in particular general patent data, classes and categories, assignees, associated companies, etc. The objective of this phase is to have a complete dataframe containing all necessary patent *metadata* for our subsequent analysis.

We start with some configuration definitions.

In [190]:
# General patent data
PATENT_DATA_URL = 'http://www.nber.org/patents/apat63_99.zip'
PATENT_DATA_FILENAME = 'apat63_99.txt'
PATENT_DATA_PATH = DATA_PATH + '/' + 'patent-data.csv'

# Patent classes
PATENT_CLASSES_URL = 'http://www.nber.org/patents/list_of_classes.txt'
PATENT_CLASSES_PATH = DATA_PATH + '/' + 'patent-classes.txt'

# Patent assignees
PATENT_ASSIGNEES_URL = 'http://www.nber.org/patents/aconame.zip'
PATENT_ASSIGNEES_FILENAME = 'aconame.txt'
PATENT_ASSIGNEES_PATH = DATA_PATH + '/' + 'patent-assignees.csv'

# Patent CUSIP numbers (Committee on Uniform Securities Identification Procedures)
# Essentially a mapping between assignees and company/parent company/identifiers
PATENT_CUSIPS_URL = 'http://www.nber.org/patents/amatch.zip'
PATENT_CUSIPS_FILENAME = 'match.csv'
PATENT_CUSIPS_PATH = DATA_PATH + '/' + 'patent-cusips.csv'

We start by fetching the compressed data file from the NBER's website, which we extract into our data directory, and rename accordingly.

In [111]:
response = urllib.request.urlopen(PATENT_DATA_URL)
compressed_file = io.BytesIO(response.read())
decompressed_file = zipfile.ZipFile(compressed_file)
decompressed_file.extractall(DATA_PATH)
os.rename(DATA_PATH + '/' + PATENT_DATA_FILENAME, PATENT_DATA_PATH)

We can now load the data into a pandas dataframe and take a look...

In [125]:
patent_data = pd.read_csv(PATENT_DATA_PATH)
patent_data.columns = map(str.lower, patent_data.columns)
patent_data

,patent,gyear,gdate,appyear,country,postate,assignee,asscode,claims,nclass,...,creceive,ratiocit,general,original,fwdaplag,bckgtlag,selfctub,selfctlb,secdupbd,secdlwbd
0,3070801,1963,1096,NaN,BE,NaN,NaN,1,NaN,269,...,1,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3070802,1963,1096,NaN,US,TX,NaN,1,NaN,2,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3070803,1963,1096,NaN,US,IL,NaN,1,NaN,2,...,9,NaN,0.3704,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3070804,1963,1096,NaN,US,OH,NaN,1,NaN,2,...,3,NaN,0.6667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3070805,1963,1096,NaN,US,CA,NaN,1,NaN,2,...,1,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3070806,1963,1096,NaN,US,PA,NaN,1,NaN,2,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3070807,1963,1096,NaN,US,OH,NaN,1,NaN,623,...,3,NaN,0.4444,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3070808,1963,1096,NaN,US,IA,NaN,1,NaN,623,...,4,NaN,0.3750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3070809,1963,1096,NaN,US,AZ,NaN,1,NaN,4,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3070810,1963,1096,NaN,US,IL,NaN,1,NaN,4,...,3,NaN,0.4444,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The above is a very good start, but it is not sufficient for our purposes. In particular, patent assignees and patent classes, categories, etc. are all identified by numbers. We now seek to enrich that data using additional mappings provided by the NBER. This enrichment will allow us to obtain a dataframe containing patent class and category names, assignee names and company information.

We begin by fetching the classes mapping, which we write into our data folder.

In [113]:
response = urllib.request.urlopen(PATENT_CLASSES_URL)
file = io.BytesIO(response.read())

with open(PATENT_CLASSES_PATH, 'wb') as outfile:
    outfile.write(file.read())

We can now load this file into a dataframe and see what it looks like. Since this is a text file with tab-separated columns and a few lines of of description at the beginning, we load the data with pandas' CSV parser using a tab character separator (`\t`) and skip the unwanted rows.

In [114]:
patent_classes = pd.read_csv(PATENT_CLASSES_PATH, sep='\t', skiprows=list(range(0,7)) + [8])
patent_classes.columns = ['nclass', 'nclass_title']
patent_classes

,nclass,nclass_title
0,1,** Classification Undetermined **
1,2,Apparel
2,4,"Baths, Closets, Sinks, and Spittoons"
3,5,Beds
4,7,Compound Tools
5,8,Bleaching and Dyeing; Fluid Treatment and Chem...
6,12,Boot and Shoe Making
7,14,Bridges
8,15,"Brushing, Scrubbing, and General Cleaning"
9,16,Miscellaneous Hardware


This is basically a simple mapping from class identifier to class name.

We now move on to the patent assignees. As before, we collect the data from NBER's website. This time, the data is in CSV format, contained within a ZIP.

In [115]:
response = urllib.request.urlopen(PATENT_ASSIGNEES_URL)
compressed_file = io.BytesIO(response.read())
decompressed_file = zipfile.ZipFile(compressed_file)
decompressed_file.extractall(DATA_PATH)
os.rename(DATA_PATH + '/' + PATENT_ASSIGNEES_FILENAME, PATENT_ASSIGNEES_PATH)

We can now load the corresponding CSV and observe the mapping.

In [116]:
patent_assignees = pd.read_csv(PATENT_ASSIGNEES_PATH)
patent_assignees.columns = map(str.lower, patent_assignees.columns)
patent_assignees

,assignee,compname
0,5,"AAA ASSOCIATES, INC."
1,10,"AAA OFFICE COFFEE SERVICE, INC."
2,15,AAA PIPE CLEANING CORPORATION
3,20,AAA PRODUCTS INTERNATIONAL INC.
4,25,"AAA SALES & ENGINEERING, INC."
5,30,AAA STEEL AND ENTERPRISES CORP.
6,35,AA. BROVIG A.S.
7,40,AACCURATE COUNTERS COMPANY
8,50,A. A. COMPUTERIZED SECURITY DOORS 1989 LTD.
9,55,"AADAMS MERCHANDISING, INC."


As for classes, this is a simple map from assignee identifier to assignee name.

Finally, we move on to the CUSIP patent dataset. This dataset contains additional details on patent assignees, specifically company name, company identifier (CUSIP), and parent company. As for the assignee data, this is CSV file compressed in a ZIP file.

In [117]:
response = urllib.request.urlopen(PATENT_CUSIPS_URL)
compressed_file = io.BytesIO(response.read())
decompressed_file = zipfile.ZipFile(compressed_file)
decompressed_file.extractall(DATA_PATH)
os.rename(DATA_PATH + '/' + PATENT_CUSIPS_FILENAME, PATENT_CUSIPS_PATH)

We load it to see what it looks like.

In [118]:
patent_cusips = pd.read_csv(PATENT_CUSIPS_PATH)
patent_cusips.columns = map(str.lower, patent_cusips.columns)
patent_cusips

,assignee,assname,cname,cusip,own,pname,sname
0,105,AAI CORPORATION,UNITED INDUSTRIAL CORP,910671,NaN,NaN,NaN
1,220,AAR CORP,AAR CORP,000361,NaN,NaN,NaN
2,365,AB AG LABORATORIES LIMITED,DEL LABORATORIES INC,245091,NaN,NaN,NaN
3,735,ABBOTT LABORATORIES,ABBOTT LABORATORIES,002824,NaN,NaN,NaN
4,785,ABB POWER T & D CPY INC,WESTINGHOUSE ELECTRIC CORP,960402,55.0,WESTINGHOUSE ELECTRIC CORP,ABB POWER T & D CO
5,860,ABB VETCOGRAY INC.,NaN,NaN,NaN,NaN,NaN
6,1045,A B CHANCE CPY,EMERSON ELECTRIC CO,291011,100.0,EMERSON ELECTRIC CO,A B CHANCE CO
7,1175,A B DICK CPY,DICK (A.B.) CO,253034,NaN,NaN,NaN
8,1450,ABEX CORPORATION,"ABEX, INC",003387,NaN,NaN,NaN
9,1850,ABITIBI PRICE CORPORATION,ABITIBI PRICE INC,003680,NaN,NaN,NaN


This is simply a map between assignee identifier and tuples containing assignee details. We notice that this dataset is significantly smaller than the assignee dataset mapping identifier to name. Perhaps this is due to the difficulty of collecting such information about companies. It is also possible that various unlisted companies do not appear in this list (since they do not possess a CUSIP), while still potentially holding patents.

We can proceed to merge the data, joining on the attributes of interest (`nclass` for patent class, `assignee` for patent assignees). We use inner joins for patent classes and assignee names, but a left join for CUSIPs (in the hope of preserving some salvageable data).

Once the data is merged, we index it by patent number and sort it for good measure.

In [192]:
patents = patent_data.merge(patent_classes).merge(patent_assignees).merge(patent_cusips, how='left')
patents.set_index(['patent'], inplace=True)
patents.sort_index(inplace=True)
patents

,gyear,gdate,appyear,country,postate,assignee,asscode,claims,nclass,cat,...,secdupbd,secdlwbd,nclass_title,compname,assname,cname,cusip,own,pname,sname
patent,,,,,,,,,,,,,,,,,,,,,
3397088,1968,3147,1964.0,US,CA,361390,2,NaN,429,4,...,0.0000,0.0000,Chemistry: Electrical Current Producing Appar...,MCDONNELL DOUGLAS CORP.,MCDONNELL DOUGLAS CORP,MCDONNELL DOUGLAS CORP,580169,NaN,NaN,NaN
3419907,1969,3294,1966.0,DE,NaN,498570,3,NaN,2,6,...,0.0000,0.0000,Apparel,SCHUBERTH-WERK K.G.,NaN,NaN,NaN,NaN,NaN,NaN
3419911,1969,3294,1966.0,US,PA,26640,2,NaN,4,6,...,0.0000,0.0000,"Baths, Closets, Sinks, and Spittoons",AMERICAN STERILIZER COMPANY,AMERICAN STERILIZER CPY,AMERICAN STERILIZER CO,030087,NaN,NaN,NaN
3419915,1969,3294,1966.0,US,CT,218550,2,NaN,4,6,...,0.0000,0.0000,"Baths, Closets, Sinks, and Spittoons",GENERAL ELECTRIC COMPANY,GENERAL ELECTRIC CPY,GENERAL ELECTRIC CO-PRE FASB,36999Z,NaN,NaN,NaN
3419915,1969,3294,1966.0,US,CT,218550,2,NaN,4,6,...,0.0000,0.0000,"Baths, Closets, Sinks, and Spittoons",GENERAL ELECTRIC COMPANY,GENERAL ELECTRIC COMPANY,GENERAL ELECTRIC CO,369604,NaN,NaN,NaN
3419919,1969,3294,1966.0,US,IL,191235,2,NaN,4,6,...,0.0000,0.0000,"Baths, Closets, Sinks, and Spittoons",FEDERAL-HUBER CORPORATION,NaN,NaN,NaN,NaN,NaN,NaN
3419922,1969,3294,1966.0,US,CA,184725,2,NaN,5,6,...,0.0000,0.0000,Beds,"EVEREST + JENNINGS, INC.",EVEREST & JENNINGS INC,EVEREST & JENNINGS -CL A,299767,NaN,NaN,NaN
3419925,1969,3294,1966.0,CA,NaN,150560,3,NaN,8,1,...,0.0000,0.0000,Bleaching and Dyeing; Fluid Treatment and Chem...,DOBBIE INDUSTRIES LIMITED,NaN,NaN,NaN,NaN,NaN,NaN
3419929,1969,3294,1967.0,US,MA,544845,2,NaN,12,6,...,0.0000,0.0000,Boot and Shoe Making,"STETSON SHOE, AND BATCHELDER-RUBICO, INC.",NaN,NaN,NaN,NaN,NaN,NaN


We now have a nice little dataset containing various patent (meta)data that we can work with.

### Stanford Patent Citation Graph

The second dataset of interest for this project is the patent citation graph from Stanford University. This graph was built using patents from 1963 to 1999. Although we would have wished to go all the way to 2017, this is already a pretty comprehensive dataset.

This dataset takes the form of a graph whose nodes correspond to patent, and who edges represent patent citations.

We start by downloading the graph data from the Stanford website. The data is structured as an edge list in NET format, i.e. source vertex $\rightarrow$ target vertex (the source cites the target). It is compressed using GZIP, so we extract it and write the output file in out data folder.

In [194]:
CITATION_GRAPH_URL = 'http://snap.stanford.edu/data/cit-Patents.txt.gz'
CITATION_GRAPH_PATH = DATA_PATH + '/' + 'patent-citations.edgefile'

In [4]:
response = urllib.request.urlopen(CITATION_GRAPH_URL)
compressed_file = io.BytesIO(response.read())
decompressed_file = gzip.GzipFile(fileobj=compressed_file)

with open(CITATION_GRAPH_PATH, 'wb') as outfile:
    outfile.write(decompressed_file.read())

Pandas is not the best suited option to work with graph-structured data. If we had a very large graph, we could use analytics frameworks such as GraphX or Chaos (system built by a team member). However, the graph input is a mere hundreds of megabytes, so we can directly use python, specifically the `networkx` package for convience.

We begin by loading the edgelist, creating a ***directed graph***.

In [195]:
G = nx.read_edgelist(CITATION_GRAPH_PATH, create_using=nx.DiGraph(), nodetype=int)
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 3774768
Number of edges: 16518948
Average in degree:   4.3761
Average out degree:   4.3761


As we can see, the graph has almost 4 million vertices and 16 millions edges.

### S&P 500 Components & Valuation Dataset

The third component of our analysis corresponds to actual market information that we can use to correlate with patent information, specifically assignees.

We opt to use the ***Standard & Poor's 500*** (S&P 500) stock market index, which is based on the market capitalizations of 500 large companies. This allows us to look at the world's largest companies and compare patents granted to these companies with their market capitalizations.

We can build our own market dataset by collecting information on the top 10 components (companies) in the S&P 500. Specifically, it is relatively easy to parse the contents of [this webpage](http://etfdb.com/history-of-the-s-and-p-500) to collect the top 10 for each year since 1980 along with each company's market capitalization (in M$).

We use `BeautifulSoup` along with `requests` to extract the relevant data.

In [182]:
SP500_TOP10_URL = 'http://etfdb.com/history-of-the-s-and-p-500'
SP500_TOP10_PATH = DATA_PATH + '/' + 'sp500_top10.csv'
SP500_START = 1980
SP500_NUM_PER_YEAR = 10

In [185]:
r = requests.get(SP500_TOP10_URL)
soup = BeautifulSoup(r.text, 'lxml')
titles = list(map(lambda e: e.get_text().strip(), soup.select('span.rowTitle')))
marketcaps = list(map(lambda e: int(e.get_text().strip().replace('$', '').replace(',', '')), soup.select('div.colFill div')))

In [186]:
assert(len(titles) == len(marketcaps)) # make sure we get the same number for both

We can now build a dataframe indexed by year and rank that contains the ranking and market capitalization of the top 10 components in the S&P 500.

In [187]:
sp500 = pd.DataFrame([(SP500_START + yearoffset, i+1, titles[yearoffset * SP500_NUM_PER_YEAR + i], marketcaps[yearoffset * SP500_NUM_PER_YEAR + i]) for i in range(0, SP500_NUM_PER_YEAR) for yearoffset in range(0,int(len(list(titles)) / SP500_NUM_PER_YEAR))])
sp500.columns = ['year', 'rank', 'company', 'marketcap']
sp500.set_index(['year', 'rank'], inplace=True)
sp500.sort_index(inplace=True)

This is what it looks like.

In [188]:
sp500

company  marketcap
year rank                                       
1980 1                            IBM      39604
     2                           AT&T      35676
     3                          Exxon      34856
     4        Standard Oil of Indiana      23365
     5                   Schlumberger      22331
     6                      Shell Oil      17990
     7                          Mobil      17163
     8     Standard Oil of California      17020
     9             Atlantic Richfield      15030
     10              General Electric      13883
1981 1                           AT&T      47750
     2                            IBM      33587
     3                          Exxon      27071
     4                   Schlumberger      16170
     5        Standard Oil of Indiana      15378
     6     Standard Oil of California      14666
     7                      Shell Oil      13594
     8               General Electric      13072
     9                 General Motors      11590
     10                 Eastman Kodak      11478
1982 1                            IBM      57794
     2                           AT&T      52989
     3                          Exxon      25863
     4               General Electric      21555
     5                 General Motors      19176
     6                  Eastman Kodak      14250
     7                   Schlumberger      13643
     8        Standard Oil of Indiana      11630
     9                      Shell Oil      11433
     10    Standard Oil of California      10947
...                               ...        ...
2011 1                    Exxon Mobil     406272
     2                          Apple     376411
     3                      Microsoft     218380
     4                            IBM     216724
     5                        Chevron     211894
     6                         Google     209199
     7                Wal-Mart Stores     204660
     8               General Electric     189082
     9             Berkshire Hathaway     188929
     10              Procter & Gamble     183541
2012 1                          Apple     498275
     2                    Exxon Mobil     403867
     3                      Microsoft     228170
     4               General Electric     226714
     5                            IBM     216936
     6                        Chevron     211199
     7              Johnson & Johnson     196067
     8                           AT&T     194938
     9               Procter & Gamble     191752
     10                        Pfizer     186494
2013 1                          Apple     481110
     2                    Exxon Mobil     439190
     3                         Google     381000
     4                      Microsoft     297900
     5               General Electric     275600
     6              Johnson & Johnson     265340
     7                        Chevron     236490
     8               Procter & Gamble     241300
     9                 JPMorgan Chase     220700
     10                   Wells Fargo     218100

[340 rows x 2 columns]

Since we have just collected ourselves a nice little dataset, let's save it to a CSV file.

In [189]:
sp500.to_csv(SP500_TOP10_PATH)

## Analysis

TBW